In [ ]:
from pyfitit import *
initPyfitit()

# Define molecule structure by given geometry parameters
### Note in python all numeration starts from 0.

In [ ]:
def moleculeConstructor(project, params):
    projectFolder = getProjectFolder()
    
    ### specify name of your initial xyz file and atomic numbers for each part that will move
    
    m = Molecule(join(projectFolder,'Fe_terpy.xyz'))
    m.setParts('0','1-9','10-19','20-29','30-38','39-48','49-58')
    
    ### specify each deformation. Note that all numeration in Python starts from 0.
    #1. Name of the deformation i.e. name of geometry parameter
    #2. Parts of the stucture involved
    #3. Axis for deformation, which passes through two atoms
    #4. Move atoms
    
    deformation = 'centralRing1_Shift'
    part = 1
    axis = m.atom[1] - m.atom[0]; axis = axis / norm(axis)
    m.part[part].shift(axis*params[deformation])
    
    deformation = 'sideRings1_Shift'
    part1 = 2; part2 = 3
    axis = m.atom[1] - m.atom[0]; axis = axis / norm(axis)
    m.part[part1].shift(axis*params[deformation])
    m.part[part2].shift(axis*params[deformation])
    
    deformation = 'sideRings1_Elong'
    part1 = 2; part2 = 3
    axis1 = m.atom[10] - m.atom[0]; axis1 = axis1 / norm(axis1)
    axis2 = m.atom[20] - m.atom[0]; axis2 = axis2 / norm(axis2)
    m.part[part1].shift(axis1*params[deformation])
    m.part[part2].shift(axis2*params[deformation])
    
    deformation = 'centralRing2_Shift'
    part = 4
    axis = m.atom[30] - m.atom[0]; axis = axis / norm(axis)
    m.part[part].shift(axis*params[deformation])
    
    deformation = 'sideRings2_Shift'
    part1 = 5; part2 = 6
    axis = m.atom[30] - m.atom[0]; axis = axis / norm(axis)
    m.part[part1].shift(axis*params[deformation])
    m.part[part2].shift(axis*params[deformation])
    
    deformation = 'sideRings2_Elong'
    part1 = 5; part2 = 6
    axis1 = m.atom[39] - m.atom[0]; axis1 = axis1 / norm(axis1)
    axis2 = m.atom[49] - m.atom[0]; axis2 = axis2 / norm(axis2)
    m.part[part1].shift(axis1*params[deformation])
    m.part[part2].shift(axis2*params[deformation])
    
    # return molecule after check
    if not m.checkInteratomicDistance(minDist = 0.8):
        print('Warning: there are atoms with distance < minDist')
    return m


# Note in python all numeration starts from 0.

In [ ]:
#specify the filename for the experimental data
def projectConstructor(expFile='exp_ground.txt'):
    project = Project()
        
    project.name = 'Feterpy_'+expFile
    project.expFile = expFile
    
    filePath = join(getProjectFolder(), expFile)
    #load experimental data. 
    #Specify energy column number (numbering starts from zero)
    #Specify intensity column number
    #Specify number of lines to skip: skiprows = 1
    project.spectrum = readSpectrum(filePath, energyColumn=0, intensityColumn=1, skiprows = 1)
    #Number of spectrum points to use for machine learning prediction (bigger is slower)
    project.maxSpectrumPoints = 100
    
    #specify part of experimental spectrum for fitting
    a = 7113; b = 7213
    project.intervals = {
      'fit_norm': [a, b],
      'fit_smooth': [a, b],
      'fit_geometry': [a, b],
      'plot': [a, b]
    }
    #specify ranges of deformations
    project.geometryParamRanges = {
        'centralRing1_Shift': [-0.3, 0.5], 
        'sideRings1_Shift': [-0.3, 0.5], 
        'sideRings1_Elong': [-0.3, 0.5], 
        'centralRing2_Shift': [-0.3, 0.5], 
        'sideRings2_Shift': [-0.3, 0.5], 
        'sideRings2_Elong': [-0.3, 0.5]
    }
    #specify parameters of calculation by FDMNES
    project.FDMNES_calc = {
        'Energy range': '-15 0.02 8 0.1 18 0.5 30 2 54 3 117',
        'Green': False,
        'radius': 5,
    }
    #specify convolution parameters and spectrum shift
    project.FDMNES_smooth = {
        'Gamma_hole': 1.5,
        'Ecent': 50,
        'Elarg': 50,
        'Gamma_max': 15,
        'Efermi': 0,
        'shift': 7113,
    }
    project.FEFF_calc = {
        'temperature': 300, 
        'Debye Temperature': 300,
    }
    project.moleculeConstructor = MethodType(moleculeConstructor, project)
    return project
    

==========================================================================================

Anything below this line won't be included into the resulting project file. You can write arbitrary code from now on. Call projectConstructor for the given project parameters and then calls moleculaConstructor for some geometry parameters.

In [ ]:
checkProject(projectConstructor, checkProjectParameters = {'expFile':'exp_ground.txt'})

Check the file molecule.xyz created by command checkproject with specified structural parameters. You can specify any set of parameters to check your deformations

In [ ]:
saveNotebook()

This command must be executed to prepare input file for successive calculations

In [ ]:
saveAsProject('FeterpyProject.py')